In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import labrotation.file_handling as fh
import h5py
from time import time
import matplotlib.pyplot as plt
import numpy as np
import os
from labrotation import file_handling as fh
from copy import deepcopy
import pandas as pd
import labrotation.two_photon_session as tps
import seaborn as sns
import uuid  # for unique labeling of sessions and coupling arrays (mouse velocity, distance, ...) to sessions in dataframe 
from matplotlib import cm  # colormap
import datadoc_util
from labrotation import two_photon_session as tps
from datetime import datetime
import seaborn as sns
from math import floor, ceil
from scipy.stats import ttest_ind, ttest_rel, mannwhitneyu, ranksums, normaltest
from numpy.polynomial.polynomial import Polynomial

In [ ]:
sns.set(font_scale=2)
sns.set_style("whitegrid")

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
N_AM_FRAMES_TO_FIND_DARKEST = 5000  # stick to old, shorter traces for finding darkest time point

In [ ]:
save_dsets = True

In [ ]:
save_figs = True
save_as_eps = False
save_as_pdf = True
if save_as_eps:
    output_format = ".eps"
elif save_as_pdf:
    output_format=".pdf"
else:
    output_format = ".jpg"
if save_figs:
    print(output_format)

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict.keys():
    docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    docu_folder = fh.open_dir("Choose folder containing folders for each mouse!")
print(f"Selected folder:\n\t{docu_folder}")

In [ ]:
if "documentation" in os.listdir(docu_folder):
    mouse_folder = os.path.join(docu_folder, "documentation")
else:
    mouse_folder = docu_folder
mouse_names = os.listdir(mouse_folder)
print(f"Mice detected:")
for mouse in mouse_names:
    print(f"\t{mouse}")

In [ ]:
def get_datetime_for_fname():
    now = datetime.now()
    return f"{now.year:04d}{now.month:02d}{now.day:02d}-{now.hour:02d}{now.minute:02d}{now.second:02d}"

In [ ]:
output_folder = env_dict["DOWNLOADS_FOLDER"]
print(f"Output files will be saved to {output_folder}")

In [ ]:
ddoc = datadoc_util.DataDocumentation(docu_folder)
ddoc.loadDataDoc()

## Load all seizures dataset

In [ ]:
df_events = ddoc.getEventsDf()
df_events = df_events[df_events["event_type"] == "sz"] 

In [ ]:
event_traces_fpath = fh.open_file("Open .h5 file containing assembled traces for all seizures!")
print(event_traces_fpath)

In [ ]:
df_colors = ddoc.getColorings()

In [ ]:
traces_ca1 = []
traces_nc = []

mouse_ids_ca1 = []
mouse_ids_nc = []

uuids_ca1 = []
uuids_nc = []

session_uuids_ca1 = []
session_uuids_nc = []

recording_break_points_ca1 = []
recording_break_points_nc = []

assure_fixed_length_bl_am = False
if assure_fixed_length_bl_am:
    N_BL_FRAMES = 5000
    N_AM_FRAMES = 5000
else:
    N_BL_FRAMES = None
    N_AM_FRAMES = None

cut_bl = True
if cut_bl:
    N_BL_FRAMES = 5000  # make sure all BL are same length
    
n_bl_frames_ca1 = []
n_am_frames_ca1 = []
n_sz_frames_ca1 = []

n_bl_frames_nc = []
n_am_frames_nc = []
n_sz_frames_nc = []

N_BL_AM_FRAMES_MINIMUM = 4800    

# first keys are event uuids, inside the following dataset names:
# 'lfp_mov_t', 'lfp_mov_y', 'lfp_t', 'lfp_y', 'lv_dist', 'lv_rounds', 
# 'lv_running', 'lv_speed', 'lv_t_s', 'lv_totdist', 'mean_fluo'
with h5py.File(event_traces_fpath, "r") as hf:
    for uuid in hf.keys():
        win_type = hf[uuid].attrs["window_type"]
        mean_fluo = np.array(hf[uuid]["mean_fluo"])
        n_bl_frames = hf[uuid].attrs["n_bl_frames"]
        n_am_frames = hf[uuid].attrs["n_am_frames"]
        if "n_sz_frames" in hf[uuid].attrs.keys():
            n_sz_frames = hf[uuid].attrs["n_sz_frames"]
        else:
            n_frames = hf[uuid].attrs["n_frames"]
            n_sz_frames = n_frames - n_bl_frames - n_am_frames
        if min(n_bl_frames, n_am_frames) < N_BL_AM_FRAMES_MINIMUM:
            print(f"Skipping {uuid}: not enough bl or am frames ({n_bl_frames}, {n_am_frames})")
            continue
        else:
            if cut_bl:
                assert N_BL_FRAMES <= n_bl_frames
                mean_fluo = mean_fluo[n_bl_frames - N_BL_FRAMES:]  # cut out first elements such that 
                n_bl_frames = N_BL_FRAMES
            if assure_fixed_length_bl_am:
                assert N_BL_FRAMES == hf[uuid].attrs["n_bl_frames"]
                assert N_AM_FRAMES == hf[uuid].attrs["n_am_frames"]
            mouse_id = hf[uuid].attrs["mouse_id"]
            if win_type == "Cx":
                traces_nc.append(mean_fluo)
                uuids_nc.append(uuid)
                session_uuids_nc.append(hf[uuid].attrs["session_uuids"])
                recording_break_points_nc.append(hf[uuid].attrs["recording_break_points"])
                mouse_ids_nc.append(mouse_id)
                n_bl_frames_nc.append(n_bl_frames)
                n_am_frames_nc.append(n_am_frames)
                n_sz_frames_nc.append(n_sz_frames)
            elif win_type == "CA1":
                traces_ca1.append(mean_fluo)
                uuids_ca1.append(uuid)
                session_uuids_ca1.append(hf[uuid].attrs["session_uuids"])
                recording_break_points_ca1.append(hf[uuid].attrs["recording_break_points"])
                mouse_ids_ca1.append(mouse_id)
                n_bl_frames_ca1.append(n_bl_frames)
                n_am_frames_ca1.append(n_am_frames)
                n_sz_frames_ca1.append(n_sz_frames)
            else:
                print(f"{win_type} not recognized window type")

In [ ]:
colors_ca1 = [df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0] for mouse_id in mouse_ids_ca1]
colors_nc = [df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0] for mouse_id in mouse_ids_nc]

# Analysis

## 1. Recovery time points

In [ ]:
# get a list of trace indices that are sorted by mouse
event_uuid_mouse_id_i_trace_ca1 = []  # list of (event_uuid, mouse_id, i_trace) tuples
event_uuid_mouse_id_i_trace_nc  = []  # list of (event_uuid, mouse_id, i_trace) tuples


for event_uuid in df_events["event_uuid"].unique():
    mouse_id = df_events[df_events["event_uuid"] == event_uuid].mouse_id.iloc[0]
    if event_uuid in uuids_ca1:
        i_trace = uuids_ca1.index(event_uuid)
        event_uuid_mouse_id_i_trace_ca1.append((event_uuid, mouse_id, i_trace))
    elif event_uuid in uuids_nc:
        i_trace = uuids_nc.index(event_uuid)
        event_uuid_mouse_id_i_trace_nc.append((event_uuid, mouse_id, i_trace))
        
    else:
        print(f"Unknown event_uuid: {event_uuid}")

In [ ]:
window_width_s = 10
window_step_s = 5
imaging_frequency = 15. # in Hz
n_frames_before_nc = 200  # include 200 frames just before aftermath for NC recordings  
n_frames_before_ca1 = 0
n_windows_post_darkest = 300 #40 # dataset consists of bl, darkest point, and this many windows post darkest point

default_bl_center_ca1 = 4925
default_bl_center_nc = 4025

window_width_frames = int(window_width_s*imaging_frequency)
window_step_frames = int(window_step_s*imaging_frequency)

half_window_width_frames = window_width_frames//2

# define baseline windows
bl_windows_nc = [(default_bl_center_nc - half_window_width_frames, default_bl_center_nc + half_window_width_frames) for i in range(len(traces_nc))]  # for neocortex, allow for ~1 min before Sz (LFP sz comes earlier)
bl_windows_ca1 = [(default_bl_center_ca1 - half_window_width_frames, default_bl_center_ca1 + half_window_width_frames) for i in range(len(traces_ca1))]  # for CA1, immediately before Sz onset

i_frame_begin_bl = 3850  # in 0-indexing, the first frame to be included in baseline
i_frame_end_bl = 4000  # in 0-indexing, the first frame after baseline (i.e. not included)

time_points = ["bl", "darkest"] + [f"{(i+1)*window_step_s}s" for i in range(n_windows_post_darkest)]
time_points_numeric = [-window_step_s, 0] + [(i+1)*window_step_s for i in range(n_windows_post_darkest)]
time_points_numeric = np.array(time_points_numeric)


def get_metric_for_window(trace_window):
    lowest_5p_indices = np.argsort(trace_window)[:int(0.05*len(trace_window))]
    lowest_5p = trace_window[lowest_5p_indices]
    return np.median(lowest_5p)

def get_recovery_data(complete_trace, i_frame_begin_bl, i_frame_end_bl, n_am_frames, n_frames_before_am=0, n_frames_for_darkest=5000):
    # n_frames_before_am: for NC, need to include a few frames before the segment "aftermath" begins, due to mistakes in 
    # n_frames_for_darkest: starting with the first frame of aftermath (i.e. ignoring the extra n_frames_before_am), use this many frames to find the darkest point.
    # manual classification. In CA1, this is not necessary
    
    metrics_list = []
    x_list = []
    
    
    # The complete trace should consist of 5000 bl, x Sz, 5000 am frames.
    # get bl as just before Sz begin
    bl_trace = complete_trace[i_frame_begin_bl:i_frame_end_bl]
    x_bl = (i_frame_begin_bl + i_frame_end_bl)//2  # TODO: assign proper x
    y_bl = get_metric_for_window(bl_trace)
    
    # add bl to dataset
    x_list.append(x_bl)
    metrics_list.append(y_bl)
    
    # get am 5p darkest points
    sorted_indices = np.argsort(complete_trace)  # this cut should not influence the index
    sorted_am_indices = sorted_indices[(sorted_indices > len(complete_trace) - n_am_frames - n_frames_before_am) & (sorted_indices < len(complete_trace) - n_am_frames +  n_frames_for_darkest)]
    am_x_5p_lowest = sorted_am_indices[:int(0.05*(n_am_frames+n_frames_before_am))] 
    
    # get single coordinate for darkest part
    # find darkest 5p, take earliest 50 of them, get median frame index of these, round down to integer frame
    x_am_darkest = int(floor(np.median(np.sort(am_x_5p_lowest)[:50])))
    
    # create sliding windows, calculate metric
    for i_window in range(n_windows_post_darkest+1):  # window around darkest point + n_windows_post_darkest windows
        x_val = x_am_darkest + i_window*window_step_frames
        window_half_width = window_width_frames//2
        window_trace = complete_trace[x_val - window_half_width : x_val + window_half_width]
        y_val = get_metric_for_window(window_trace)
        x_list.append(x_val)
        metrics_list.append(y_val)
        
    return (x_list, metrics_list)
    

### Manually modify baseline values

In [ ]:
event_uuid_mouse_id_i_trace_ca1
# CA1:
# aa66ae0470a14eb08e9bcadedc34ef64 : ~4250
# c7b29d28248e493eab02288b85e3adee : 4000
# 7b9c17d8a1b0416daf65621680848b6a : 4050
# 9e75d7135137444492d104c461ddcaac : 4700
# d158cd12ad77489a827dab1173a933f9 : 4500
# a39ed3a880c54f798eff250911f1c92f : 4500
# 4e2310d2dde845b0908519b7196080e8 : 4500
# f0442bebcd1a4291a8d0559eb47df08e : 4500
# NC:
# 2251bba132cf45fa839d3214d1651392 : 3700
# cd3c1e0e3c284a89891d2e4d9a7461f4 : 3500
dict_uuid_manual_bl_center = {"aa66ae0470a14eb08e9bcadedc34ef64": 4250, "c7b29d28248e493eab02288b85e3adee": 4000,  "7b9c17d8a1b0416daf65621680848b6a": 4050, "9e75d7135137444492d104c461ddcaac": 4700, "d158cd12ad77489a827dab1173a933f9": 4500, "a39ed3a880c54f798eff250911f1c92f" : 4500, "4e2310d2dde845b0908519b7196080e8" : 4500, "f0442bebcd1a4291a8d0559eb47df08e": 4500, "2251bba132cf45fa839d3214d1651392": 3700, "cd3c1e0e3c284a89891d2e4d9a7461f4": 3500}

# map uuid : bl centre dictionary i_trace : bl_centre dict
dict_itrace_manual_bl_center_ca1 = {}
dict_itrace_manual_bl_center_nc = {}

for uuid_mouseid_itrace in event_uuid_mouse_id_i_trace_ca1:
    uuid = uuid_mouseid_itrace[0]
    mouse_id = uuid_mouseid_itrace[1]
    i_trace = uuid_mouseid_itrace[2]
    
    if uuid in dict_uuid_manual_bl_center.keys():
        dict_itrace_manual_bl_center_ca1[i_trace] = dict_uuid_manual_bl_center[uuid]
for uuid_mouseid_itrace in event_uuid_mouse_id_i_trace_nc:
    uuid = uuid_mouseid_itrace[0]
    mouse_id = uuid_mouseid_itrace[1]
    i_trace = uuid_mouseid_itrace[2]
    
    if uuid in dict_uuid_manual_bl_center.keys():
        dict_itrace_manual_bl_center_nc[i_trace] = dict_uuid_manual_bl_center[uuid]
        
# re-calculate bl_windows_ca1, bl_windows_nc
for i_trace in dict_itrace_manual_bl_center_ca1.keys():
    bl_windows_ca1[i_trace] = (dict_itrace_manual_bl_center_ca1[i_trace] - half_window_width_frames, dict_itrace_manual_bl_center_ca1[i_trace] + half_window_width_frames)
for i_trace in dict_itrace_manual_bl_center_nc.keys():
    bl_windows_nc[i_trace] = (dict_itrace_manual_bl_center_nc[i_trace] - half_window_width_frames, dict_itrace_manual_bl_center_nc[i_trace] + half_window_width_frames)

## Calculate metrics for all data

In [ ]:
x_recovery_ca1 = [[] for i in range(len(traces_ca1))]
y_recovery_ca1 = [[] for i in range(len(traces_ca1))]

x_recovery_nc = [[] for i in range(len(traces_nc))]
y_recovery_nc = [[] for i in range(len(traces_nc))]


for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_nc:
    n_bl_frames = n_bl_frames_nc[i_trace]
    n_sz_frames = n_sz_frames_nc[i_trace]
    n_am_frames = n_am_frames_nc[i_trace]# N_AM_FRAMES_TO_FIND_DARKEST  # limit to this many frames
    x_data, y_data = get_recovery_data(traces_nc[i_trace], bl_windows_nc[i_trace][0], bl_windows_nc[i_trace][1], n_am_frames_nc[i_trace], n_frames_before_nc)
    x_recovery_nc[i_trace] = x_data
    y_recovery_nc[i_trace] = y_data

for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_ca1:
    n_bl_frames = n_bl_frames_ca1[i_trace]
    n_sz_frames = n_sz_frames_ca1[i_trace]
    n_am_frames_cut = n_am_frames_ca1[i_trace]#N_AM_FRAMES_TO_FIND_DARKEST  # limit to this many frames
    x_data, y_data = get_recovery_data(traces_ca1[i_trace], bl_windows_ca1[i_trace][0], bl_windows_ca1[i_trace][1], n_am_frames_ca1[i_trace], n_frames_before_ca1)
    x_recovery_ca1[i_trace] = x_data
    y_recovery_ca1[i_trace] = y_data
    
x_recovery_ca1 = np.array(x_recovery_ca1, dtype=np.int16)
x_recovery_nc = np.array(x_recovery_nc, dtype=np.int16)

y_recovery_ca1 = np.array(y_recovery_ca1,)
y_recovery_nc = np.array(y_recovery_nc, )


### Create list of mouse id/event uuid

In [ ]:
i_recovery_nc = []
i_recovery_ca1 = []

recovery_ratio = 0.95  # reach x % of baseline to be considered recovered

def get_recovery_index(recovery_trace):
    baseline = recovery_trace[0]  # bl, darkest, windows...
    if np.nanmax(recovery_trace[2:]) < recovery_ratio*baseline:  # baseline never reached
        return -1
    else:
        i_recovery = np.argmax(recovery_trace[2:] >= recovery_ratio*baseline) + 2  # shift back to index in whole trace
        return i_recovery

for recovery_trace in y_recovery_nc:
    i_recovery = get_recovery_index(recovery_trace)
    i_recovery_nc.append(i_recovery)
    
for recovery_trace in y_recovery_ca1:
    i_recovery = get_recovery_index(recovery_trace)
    i_recovery_ca1.append(i_recovery)
    
i_recovery_ca1 = np.array(i_recovery_ca1)
i_recovery_nc = np.array(i_recovery_nc)

### Create mouse id, event uuid lists matching order of traces

In [ ]:
mouse_ids_nc = ["" for i in range(len(i_recovery_nc))]
event_uuids_nc = ["" for i in range(len(i_recovery_nc))]
for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_nc:
    mouse_ids_nc[i_trace] = mouse_id
    event_uuids_nc[i_trace] = event_uuid

In [ ]:
mouse_ids_ca1 = ["" for i in range(len(i_recovery_ca1))]
event_uuids_ca1 = ["" for i in range(len(i_recovery_ca1))]
for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_ca1:
    mouse_ids_ca1[i_trace] = mouse_id
    event_uuids_ca1[i_trace] = event_uuid

### Set "recovery time point not found" value
Instead of NaN, try to extrapolate.

In [ ]:
x_recovery_single_ca1 = []
linearly_fit_ca1 = [False for i in range(len(y_recovery_ca1))]
for i_trace_ca1 in range(len(y_recovery_ca1)):
    i_recovery = i_recovery_ca1[i_trace_ca1]
    if  i_recovery == -1:
        try:
            linearly_fit_ca1[i_trace_ca1] = True
            line_fit_coeffs = Polynomial.fit(x_recovery_ca1[i_trace_ca1][2:], y_recovery_ca1[i_trace_ca1][2:], deg=1).convert().coef  # linear fit starting with point after darkest time point
            # the coefficients [a, b] from y= a + b*x.
            if line_fit_coeffs[1] <= 0:  # Check if b is non-positive -> No recovery possible
                #x_recovery_single_ca1.append(np.inf)
                x_recovery_single_ca1.append(25000)
                continue
        except np.linalg.LinAlgError as e:
            print(f"Could not fit {i_trace_ca1}: {event_uuids_ca1[i_trace_ca1]}")
            x_recovery_single_ca1.append(x_recovery_ca1[i_trace_ca1][-1])
        else:  # b>0 -> line is ascending, i.e. there will be a recovery time
            # find inverse function. We know y = a + b*x, need to have x = c + d*y, where y = <threshold>*baseline (threshold=0.95)
            # inverse is x = -a/b + (1/b)*y = a_inv + b_inv*y
            a_inv = -line_fit_coeffs[0]/line_fit_coeffs[1]
            b_inv = 1/line_fit_coeffs[1]
            baseline = y_recovery_ca1[i_trace_ca1][0]
            x_recovery = a_inv + b_inv*recovery_ratio*baseline
            x_recovery = ceil(x_recovery)
            x_recovery_single_ca1.append(x_recovery)
    else:  # recovery was found within time frame
        x_recovery = x_recovery_ca1[i_trace_ca1][i_recovery]
        x_recovery_single_ca1.append(x_recovery)
x_recovery_single_ca1 = np.array(x_recovery_single_ca1)

In [ ]:
x_recovery_single_ca1

In [ ]:
x_recovery_single_nc = []
for i_trace_nc in range(len(y_recovery_nc)):
    i_recovery = i_recovery_nc[i_trace_nc]
    if  i_recovery == -1:
        line_fit_coeffs = Polynomial.fit(x_recovery_nc[i_trace_nc][2:], y_recovery_nc[i_trace_nc][2:], deg=1).convert().coef  # linear fit starting with point after darkest time point
        # the coefficients [a, b] from y= a + b*x.
        if line_fit_coeffs[1] <= 0:  # Check if b is non-positive -> No recovery possible
            x_recovery_single_nc.append(np.inf)
            continue
        else:  # b>0 -> line is ascending, i.e. there will be a recovery time
            # find inverse function. We know y = a + b*x, need to have x = c + d*y, where y = <threshold>*baseline (threshold=0.95)
            # inverse is x = -a/b + (1/b)*y = a_inv + b_inv*y
            a_inv = -line_fit_coeffs[0]/line_fit_coeffs[1]
            b_inv = 1/line_fit_coeffs[1]
            baseline = y_recovery_nc[i_trace_nc][0]
            x_recovery = a_inv + b_inv*recovery_ratio*baseline
            x_recovery = ceil(x_recovery)
            x_recovery_single_nc.append(x_recovery)
    else:  # recovery was found within time frame
        x_recovery = x_recovery_nc[i_trace_nc][i_recovery]
        x_recovery_single_nc.append(x_recovery)
x_recovery_single_nc = np.array(x_recovery_single_nc)

### Convert x (frame units) to t (seconds)
0 s is the center of window belonging to darkest time point

In [ ]:
def get_t_recovery(i_frame_darkest_center, i_frame_recovery):
    # Given the center of the darkest window (x_recovery_ca1[i_trace][1]) and the frame of recovery, calculate t_recovery
    # calculate distance in frames
    dframes = i_frame_recovery - i_frame_darkest_center
    # convert delta to seconds
    dt = dframes/imaging_frequency  # dt is also t1, as darkest point, t0, is 0
    
    n_windows_between = dt//window_step_s
    # check if window on left of time point contains the recovery time point
    t_window_center_left = n_windows_between*window_step_s
    t_window_center_right = (n_windows_between+1)*window_step_s
    
    t_window_edge_left = t_window_center_left + window_width_s/2
    if t_window_edge_left >= dt:  # window on left of time point contains t_recovery (dt)
        return int(t_window_center_left)
    else:  # window on right of recovery time point contains it
        return int(t_window_center_right + 1)

t_recovery_single_ca1 = []
for i_trace_ca1 in range(len(x_recovery_single_ca1)):
    i_frame_darkest = x_recovery_ca1[i_trace_ca1][1]  # center of darkest time window is 0
    i_frame_recovery = x_recovery_single_ca1[i_trace_ca1]
    t_rounded_recovery = get_t_recovery(i_frame_darkest, i_frame_recovery)
    t_recovery_single_ca1.append(t_rounded_recovery)
t_recovery_single_ca1 = np.array(t_recovery_single_ca1)

t_recovery_single_nc = []
for i_trace_nc in range(len(x_recovery_single_nc)):
    i_frame_darkest = x_recovery_nc[i_trace_nc][1]  # center of darkest time window is 0
    i_frame_recovery = x_recovery_single_nc[i_trace_nc]
    
    t_rounded_recovery = get_t_recovery(i_frame_darkest, i_frame_recovery)
    t_recovery_single_nc.append(t_rounded_recovery)
t_recovery_single_nc = np.array(t_recovery_single_nc)

In [ ]:
# add np.nan for not found onset values
#time_points_numeric_extended = np.concatenate([time_points_numeric, np.array([np.nan])])

# If no recovery found (last value in time_points_numeric_extended), assign a finite value
time_points_numeric_extended = np.concatenate([time_points_numeric, np.array([time_points_numeric[-1]+window_step_s])])

In [ ]:
df_recovery_nc = pd.DataFrame(columns=["i_recovery"],data=i_recovery_nc)
df_recovery_nc["t_recovery"] = t_recovery_single_nc#time_points_numeric_extended[i_recovery_nc]
df_recovery_nc["event_uuid"] =  event_uuids_nc
df_recovery_nc["mouse_id"] = mouse_ids_nc
df_recovery_nc["window_type"] = "nc"
#df_recovery_nc = df_recovery_nc.sort_values(by="mouse_id").reset_index().drop(columns=["index"])

df_recovery_nc["bl_center"] = np.mean(np.array(bl_windows_nc), axis=1, dtype=np.int16)
df_recovery_nc["darkest_window_center"] = x_recovery_nc.T[1] 

df_recovery_nc["window_width_frames"] = window_width_frames
df_recovery_nc["window_width_s"] = window_width_s
df_recovery_nc["window_step_frames"] = window_step_frames
df_recovery_nc["n_windows_total"] = n_windows_post_darkest + 2  # bl, darkest, n_windows_post_darkest windows

In [ ]:
df_recovery_ca1 = pd.DataFrame(columns=["i_recovery"],data=i_recovery_ca1)
df_recovery_ca1["t_recovery"] = t_recovery_single_ca1#time_points_numeric_extended[i_recovery_ca1]
df_recovery_ca1["event_uuid"] =  event_uuids_ca1
df_recovery_ca1["mouse_id"] = mouse_ids_ca1
df_recovery_ca1["window_type"] = "ca1"
df_recovery_ca1["bl_center"] = np.mean(np.array(bl_windows_ca1), axis=1, dtype=np.int16)
df_recovery_ca1["darkest_window_center"] = x_recovery_ca1.T[1] 
df_recovery_ca1["window_width_frames"] = window_width_frames
df_recovery_ca1["window_width_s"] = window_width_s
df_recovery_ca1["window_step_frames"] = window_step_frames
df_recovery_ca1["n_windows_total"] = n_windows_post_darkest + 2  # bl, darkest, n_windows_post_darkest windows
#df_recovery_ca1 = df_recovery_nc.sort_values(by="mouse_id").reset_index().drop(columns=["index"])

In [ ]:
df_recovery_combined = pd.concat([df_recovery_ca1, df_recovery_nc])

## 2. Sz-bl, SD-bl amplitude comparison

In [ ]:
sd_window_width_frames = 450  # 30 s window beginning with "am" segment to look for amplitude of SD

bl_values = y_recovery_ca1[:, 0]
sz_amplitudes_ca1 = []
sd_amplitudes_ca1 = []

# lists storing info about the sz, sd amplitude windows
begin_frames_sz_amplitude_ca1 = []
end_frames_sz_amplitude_ca1 = []
begin_frames_sd_amplitude_ca1 = []
end_frames_sd_amplitude_ca1 = []


for i_trace_ca1 in range(len(traces_ca1)):
    event_uuid = event_uuids_ca1[i_trace_ca1]
    # take whole sz for sz window; take 2 minutes directly after sz end as SD window.
    
    len_sz = len(traces_ca1[i_trace_ca1]) -  n_am_frames_ca1[i_trace_ca1] - n_bl_frames_ca1[i_trace_ca1]
    
    

    if event_uuid == "f0442bebcd1a4291a8d0559eb47df08e":  # recording(s) with no Sz, only SD
        i_begin_sd_window = len(traces_ca1[i_trace_ca1]) - n_am_frames_ca1[i_trace_ca1]
        i_end_sd_window = i_begin_sd_window + sd_window_width_frames
        
        sz_amplitudes_ca1.append(np.nan)
        sd_amplitudes_ca1.append(np.flip(np.sort(traces_ca1[i_trace_ca1][i_begin_sd_window:i_end_sd_window]))[:int(0.05*(i_end_sd_window - i_begin_sd_window))].mean())
        begin_frames_sz_amplitude_ca1.append(np.nan)
        end_frames_sz_amplitude_ca1.append(np.nan)
        begin_frames_sd_amplitude_ca1.append(i_begin_sd_window)
        end_frames_sd_amplitude_ca1.append(i_end_sd_window-1)
    else:
        i_begin_sz_window = n_bl_frames_ca1[i_trace_ca1]  # inclusive
        i_end_sz_window =  n_bl_frames_ca1[i_trace_ca1]+len_sz+1 # exclusive
        sz_amp_val_ca1 = np.flip(np.sort(traces_ca1[i_trace_ca1][i_begin_sz_window:i_end_sz_window]))[:int(0.05*( i_end_sz_window - i_begin_sz_window))].mean()  # take mean of highest 5% of sz values
        
        
        i_begin_sd_window = i_end_sz_window  # inclusive
        i_end_sd_window = i_begin_sd_window + sd_window_width_frames  # exclusive
        sd_amp_val_ca1 = np.flip(np.sort(traces_ca1[i_trace_ca1][i_begin_sd_window:i_end_sd_window]))[:int(0.05*(i_end_sd_window - i_begin_sd_window))].mean()  # take mean of highest 5% of sd values

        
        sz_amplitudes_ca1.append(sz_amp_val_ca1)
        sd_amplitudes_ca1.append(sd_amp_val_ca1)
        
        begin_frames_sz_amplitude_ca1.append(i_begin_sz_window)
        end_frames_sz_amplitude_ca1.append(i_end_sz_window-1)
        begin_frames_sd_amplitude_ca1.append(i_begin_sd_window)
        end_frames_sd_amplitude_ca1.append(i_end_sd_window-1)
    
    
# array containing three columns: baseline, sz, sd amplitudes.
bl_sz_sd_amplitudes = np.array([bl_values, sz_amplitudes_ca1, sd_amplitudes_ca1]).T

# subtract bl from sz and sd amplitudes
sz_sd_minus_bl_amplitudes = np.array([bl_sz_sd_amplitudes[:,1] - bl_sz_sd_amplitudes[:,0],  bl_sz_sd_amplitudes[:,2] - bl_sz_sd_amplitudes[:,0]]).T

In [ ]:
df_amplitudes_ca1 = pd.DataFrame(columns=["Sz-bl", "SD-bl"], data=sz_sd_minus_bl_amplitudes) # only CA1 has SD
df_amplitudes_ca1["mouse_id"] = mouse_ids_ca1  
df_amplitudes_ca1["event_uuid"] = event_uuids_ca1
df_amplitudes_ca1["window_type"] = "ca1"
df_amplitudes_ca1["first_frame_sz_window"] = begin_frames_sz_amplitude_ca1
df_amplitudes_ca1["last_frame_sz_window"] = end_frames_sz_amplitude_ca1
df_amplitudes_ca1["first_frame_sd_window"] = begin_frames_sd_amplitude_ca1
df_amplitudes_ca1["last_frame_sd_window"] = end_frames_sd_amplitude_ca1

df_amplitudes_ca1["first_frame_bl_window"] = list(zip(*bl_windows_ca1))[0]
df_amplitudes_ca1["last_frame_bl_window"] = np.array(list(zip(*bl_windows_ca1))[1]) - 1

## 3. Baseline - darkest time point difference

In [ ]:
bl_minus_darkest_ca1 = y_recovery_ca1[:,0] - y_recovery_ca1[:,1]
bl_minus_darkest_nc = y_recovery_nc[:,0] - y_recovery_nc[:,1]

In [ ]:
df_bl_minus_darkest_ca1 = pd.DataFrame(data=bl_minus_darkest_ca1, columns=["bl-darkest"])
df_bl_minus_darkest_ca1["window_type"] = "ca1"
df_bl_minus_darkest_ca1["event_uuid"] = event_uuids_ca1
df_bl_minus_darkest_ca1["mouse_id"] = mouse_ids_ca1
df_bl_minus_darkest_ca1["baseline"] = y_recovery_ca1[:,0]
df_bl_minus_darkest_ca1["darkest_postictal"] = y_recovery_ca1[:,1]

df_bl_minus_darkest_ca1["bl_center"] = np.mean(np.array(bl_windows_ca1), axis=1, dtype=np.int16)
df_bl_minus_darkest_ca1["darkest_window_center"] = x_recovery_ca1.T[1] 

df_bl_minus_darkest_ca1["window_width_frames"] = window_width_frames
df_bl_minus_darkest_ca1["window_width_s"] = window_width_s
df_bl_minus_darkest_ca1["window_step_frames"] = window_step_frames
df_bl_minus_darkest_ca1["n_windows_total"] = n_windows_post_darkest + 2  # bl, darkest, n_windows_post_darkest windows


df_bl_minus_darkest_nc = pd.DataFrame(data=bl_minus_darkest_nc, columns=["bl-darkest"])
df_bl_minus_darkest_nc["window_type"] = "nc"
df_bl_minus_darkest_nc["event_uuid"] = event_uuids_nc
df_bl_minus_darkest_nc["mouse_id"] = mouse_ids_nc
df_bl_minus_darkest_nc["baseline"] = y_recovery_nc[:,0]
df_bl_minus_darkest_nc["darkest_postictal"] = y_recovery_nc[:,1]

df_bl_minus_darkest_nc["bl_center"] = np.mean(np.array(bl_windows_nc), axis=1, dtype=np.int16)
df_bl_minus_darkest_nc["darkest_window_center"] = x_recovery_nc.T[1] 

df_bl_minus_darkest_nc["window_width_frames"] = window_width_frames
df_bl_minus_darkest_nc["window_width_s"] = window_width_s
df_bl_minus_darkest_nc["window_step_frames"] = window_step_frames
df_bl_minus_darkest_nc["n_windows_total"] = n_windows_post_darkest + 2  # bl, darkest, n_windows_post_darkest windows



df_bl_minus_darkest_combined = pd.concat([df_bl_minus_darkest_ca1, df_bl_minus_darkest_nc]) 

# Plot results

## 1. Recovery

## CA1

In [ ]:
fig = plt.figure(figsize=(18,18))
amplitude = 100.
offset = 0.

for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_ca1:
    print(f"{event_uuid} {offset}")
    c = df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0]
    plt.plot(amplitude*(traces_ca1[i_trace] - min(traces_ca1[i_trace]))/(max(traces_ca1[i_trace] - min(traces_ca1[i_trace])))+offset, color=c )
    plt.vlines(x=x_recovery_ca1[i_trace], ymin=offset-0.1*amplitude, ymax=offset+1.1*amplitude)
    for window_center in x_recovery_ca1[i_trace]:
        plt.hlines(y=offset-5, xmin=window_center-window_width_frames//2, xmax= window_center+window_width_frames//2, color="red")
    
    n_sz_frames = len(traces_ca1[i_trace]) - n_am_frames_ca1[i_trace_ca1] - n_bl_frames_ca1[i_trace]
    plt.vlines(x=[n_bl_frames_ca1[i_trace], n_bl_frames_ca1[i_trace]+n_sz_frames_ca1[i_trace], n_bl_frames_ca1[i_trace]+n_sz_frames_ca1[i_trace]+sd_window_width_frames], ymin=offset-0.1*amplitude, ymax=offset+1.1*amplitude, color="black")
    
    offset += 1.3*amplitude
plt.tight_layout()
if save_figs:
    output_fpath = os.path.join(output_folder, f"ca1_recovery_overview_{get_datetime_for_fname()}{output_format}")
    plt.savefig(output_fpath)
    print(f"Saved as {output_fpath}")
plt.show()

In [ ]:
fig = plt.figure(figsize=(24,18))
x_ticks = [i for i in range(len(y_recovery_ca1[0]))]
offset = 0.0
for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_ca1:
    c = df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0]
    plt.plot(time_points_numeric_extended[:len(y_recovery_ca1[i_trace])], y_recovery_ca1[i_trace] - y_recovery_ca1[i_trace][0] + offset, "o-", color=c, markersize=10 )
    plt.hlines(y=offset, xmin=time_points_numeric_extended[0], xmax=time_points_numeric_extended[-1], color="red")
    if linearly_fit_ca1[i_trace]:
        c = "red"
    plt.vlines(ymin=offset-5, ymax=offset+5, x=t_recovery_single_ca1[i_trace], linewidth=5, color=c)
    offset += 10
    print(f"{event_uuid}: {t_recovery_single_ca1[i_trace]}")
#plt.xticks(x_ticks, time_points)
plt.tight_layout()
if save_figs:
    output_fpath = os.path.join(output_folder, f"ca1_recovery_time_points_{get_datetime_for_fname()}{output_format}")
    plt.savefig(output_fpath)
    print(f"Saved as {output_fpath}")
#plt.xlim((100, 200))
plt.show()

In [ ]:
t_recovery_single_ca12 = t_recovery_single_ca1.copy()
y_recovery_ca12 = y_recovery_ca1.copy()

In [ ]:
t_recovery_single_nc2 = t_recovery_single_nc.copy()
y_recovery_nc2 = y_recovery_nc.copy()

In [ ]:
# TODO: 4th from bototm, for example, the recovery happens with the new recording. Need to take into account the jump in time between two recordings! So its not x seconds, but x+y, y the time between two recordings
# TODO also exclude the 4th from bottom recording, as water was evaporating.
# TODO make assembled tmev traces with 10000-10000 frames. Make manual lfp-labview matching, write down parameters.

## NC

In [ ]:
fig = plt.figure(figsize=(18,18))
amplitude = 100.
offset = 0.

for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_nc:
    #print(f"{event_uuid} {offset}")
    c = df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0]
    plt.plot(amplitude*(traces_nc[i_trace] - min(traces_nc[i_trace]))/(max(traces_nc[i_trace] - min(traces_nc[i_trace])))+offset, color=c )
    plt.vlines(x=x_recovery_nc[i_trace], ymin=offset-0.1*amplitude, ymax=offset+1.1*amplitude)
    for window_center in x_recovery_nc[i_trace]:
        plt.hlines(y=offset-5, xmin=window_center-window_width_frames//2, xmax= window_center+window_width_frames//2, color="red")
    offset += 1.3*amplitude
plt.tight_layout()
if save_figs:
    output_fpath = os.path.join(output_folder, f"nc_recovery_overview_{get_datetime_for_fname()}{output_format}")
    plt.savefig(output_fpath)
    print(f"Saved as {output_fpath}")
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,18))
offset = 0.0
for event_uuid, mouse_id, i_trace in event_uuid_mouse_id_i_trace_nc:
    c = df_colors[df_colors["mouse_id"] == mouse_id].color.iloc[0]
    plt.plot(time_points_numeric_extended[:len(y_recovery_nc[i_trace])], y_recovery_nc[i_trace] - y_recovery_nc[i_trace][0] + offset, "o-", color=c, markersize=10 )
    plt.hlines(y=offset, xmin=time_points_numeric_extended[0], xmax=time_points_numeric_extended[-1], color="red")
    plt.vlines(ymin=offset-5, ymax=offset+5, x=t_recovery_single_nc[i_trace], linewidth=5, color=c)
    offset += 10
plt.tight_layout()
if save_figs:
    output_fpath = os.path.join(output_folder, f"nc_recovery_time_points_{get_datetime_for_fname()}{output_format}")
    plt.savefig(output_fpath)
    print(f"Saved as {output_fpath}")
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8,6), sharey=True)
left = sns.barplot(x="window_type", y="t_recovery", data=df_recovery_combined, ax= axs[0])
right = sns.stripplot(x="window_type", y="t_recovery", hue="window_type", data=df_recovery_combined, ax=axs[1], s=15, legend=False)
#left.set(ylim=(0, time_points_numeric_extended[-1]+10))
#right.set(ylim=(0, 200))
plt.tight_layout()
if save_figs:
    output_fpath = os.path.join(output_folder, f"recovery_ca1_vs_nc_{get_datetime_for_fname()}{output_format}")
    plt.savefig(output_fpath)
    print(f"Saved as {output_fpath}")
plt.show()

## 2. Sz-bl, SD-bl amplitudes

In [ ]:
fig = plt.figure(figsize=(4,10))
for i_sz in range(len(sz_sd_minus_bl_amplitudes)):
    plt.plot(sz_sd_minus_bl_amplitudes[i_sz], "o-", color=colors_ca1[i_sz], )
#for i_sz_nc in range(len(amplitudes_nc_to_bl)):
#    plt.plot(amplitudes_nc_to_bl[i_sz_nc], "o-", color=colors_nc[i_sz_nc])
plt.ylabel("fluorescence (a.u.)")
    
plt.vlines([0,1], 0, 500)
    
plt.text(0.05, 450, "Sz-bl", fontdict=None)
plt.text(0.68, 450, "SD-bl", fontdict=None)

frame1 = plt.gca()
frame1.axes.xaxis.set_ticklabels([])
frame1.axes.xaxis.grid(False)
#frame1.axes.yaxis.set_ticklabels([])
plt.tight_layout()
if save_figs:
    output_fpath = os.path.join(output_folder, f"amplitudes_to_bl_{get_datetime_for_fname()}{output_format}")
    plt.savefig(output_fpath)
    print(f"Saved as {output_fpath}")
plt.show()

## 3. Bl - darkest time point difference

In [ ]:
fig, axs=plt.subplots(1,2,figsize=(8,6), sharey=True)
left= sns.barplot(data=df_bl_minus_darkest_combined, x="window_type", y="bl-darkest", ax=axs[0])
right = sns.stripplot(data=df_bl_minus_darkest_combined, x="window_type", y="bl-darkest", hue="window_type", legend=False, s=15, ax=axs[1])
plt.tight_layout()
if save_figs:
    output_fpath = os.path.join(output_folder, f"recovery_bl_minus_darkest_{get_datetime_for_fname()}{output_format}")
    plt.savefig(output_fpath)
    print(f"Saved as {output_fpath}")
plt.show()

In [ ]:
#fig = plt.figure(figsize=(18,12))
#for i_sz in range(len(amplitudes_ca1)):
#    plt.plot(amplitudes_ca1[i_sz], color=colors_ca1[i_sz])
#plt.show()

# Export datasets

# Statistical tests
t-test assumes normal distribution, which probably is not the case...

In [ ]:
output_table =  {"what_tested": [], "test_type": [], "statistic": [], "p_val": [], "comment": []}

## 1. Recovery 

In [ ]:
if save_dsets:
    output_fpath = os.path.join(output_folder, f"recovery_dataset_{get_datetime_for_fname()}.xlsx")
    df_recovery_combined[["mouse_id", "window_type", "t_recovery"]].to_excel(output_fpath, index=False)
    print(f"Saved to {output_fpath}")

In [ ]:
t_recovery_ca1 = df_recovery_combined[df_recovery_combined["window_type"] == "ca1"].t_recovery
t_recovery_nc = df_recovery_combined[df_recovery_combined["window_type"] == "nc"].t_recovery

In [ ]:
nt_rec_ca1 = normaltest(t_recovery_ca1)
print(nt_rec_ca1)

In [ ]:
# CA1
output_table["what_tested"].append("t_recovery_ca1")
output_table["test_type"].append("normality")
output_table["statistic"].append(nt_rec_ca1.statistic)
output_table["p_val"].append(nt_rec_ca1.pvalue)
if nt_rec_ca1.pvalue < 0.05:
    output_table["comment"].append("dataset normality hypothesis rejected")
else:
    output_table["comment"].append("dataset normality hypothesis was not rejected")

In [ ]:
nt_rec_nc = normaltest(t_recovery_nc)
print(nt_rec_nc)

In [ ]:
# NC
output_table["what_tested"].append("t_recovery_nc")
output_table["test_type"].append("normality")
output_table["statistic"].append(nt_rec_nc.statistic)
output_table["p_val"].append(nt_rec_nc.pvalue)
if nt_rec_nc.pvalue < 0.05:
    output_table["comment"].append("dataset normality hypothesis rejected")
else:
    output_table["comment"].append("dataset normality hypothesis not rejected")

### As for the CA1 recovery data, we must reject the null hypothesis, we should use consistently Mann-Whitney. Test for sample mean equality for CA1 vs NC

In [ ]:
# need to choose which assumptions fit the data, i.e. which test is appropriate
#print(ttest_ind(t_recovery_ca1, t_recovery_nc))
#print(ranksums(t_recovery_ca1, t_recovery_nc))
mwu_recovery = mannwhitneyu(t_recovery_ca1, t_recovery_nc)
print(mwu_recovery)

In [ ]:
output_table["what_tested"].append("t_recovery")
output_table["test_type"].append("Mann-Whitney U")
output_table["statistic"].append(mwu_recovery.statistic)
output_table["p_val"].append(mwu_recovery.pvalue)
if mwu_recovery.pvalue < 0.05:
    output_table["comment"].append("two datasets coming from same population rejected")
else:
    output_table["comment"].append("two datasets coming from same population not rejected")

## 2. Bl-Sz, Bl-SD amplitude comparison

### Test normality

### As for CA1, the null hypothesis that the sample came from normal distribution is rejected, one can use the Mann-Whitney test instead of the unpaired t-test.

In [ ]:
# sz_sd_minus_bl_amplitudes: [[Bl2-Sz1, Bl2-SD1], [Bl2-Sz2, BL2-SD2], ...]
bl_minus_sz = sz_sd_minus_bl_amplitudes.T[0]
bl_minus_sd = sz_sd_minus_bl_amplitudes.T[1]
no_nan_idx = ~(np.isnan(bl_minus_sz) | np.isnan(bl_minus_sd))

In [ ]:
# test amplitude of Sz vs amplitude of SD: from same population?

#print(ttest_ind(bl_minus_sz[no_nan_idx], bl_minus_sd[no_nan_idx]))
#print(ranksums(bl_minus_sz[no_nan_idx], bl_minus_sd[no_nan_idx]))
mwu_ampl = mannwhitneyu(bl_minus_sz[no_nan_idx], bl_minus_sd[no_nan_idx])
print(mwu_ampl)

In [ ]:
output_table["what_tested"].append("Sz_vs_SD_amplitude")
output_table["test_type"].append("Mann-Whitney U")
output_table["statistic"].append(mwu_ampl.statistic)
output_table["p_val"].append(mwu_ampl.pvalue)
if mwu_ampl.pvalue < 0.05:
    output_table["comment"].append("two datasets coming from same population rejected")
else:
    output_table["comment"].append("two datasets coming from same population not rejected")

## 3. Bl - darkest time point difference

In [ ]:
nt_diff_ca1 = normaltest(df_bl_minus_darkest_combined[df_bl_minus_darkest_combined["window_type"] == "ca1"]["bl-darkest"])
print(nt_diff_ca1)

In [ ]:
# CA1
output_table["what_tested"].append("bl_minus_darkest_ca1")
output_table["test_type"].append("normality")
output_table["statistic"].append(nt_diff_ca1.statistic)
output_table["p_val"].append(nt_diff_ca1.pvalue)
if nt_diff_ca1.pvalue < 0.05:
    output_table["comment"].append("dataset normality hypothesis rejected")
else:
    output_table["comment"].append("dataset normality hypothesis was not rejected")

In [ ]:
nt_diff_nc = normaltest(df_bl_minus_darkest_combined[df_bl_minus_darkest_combined["window_type"] == "nc"]["bl-darkest"])
print(nt_diff_nc)

In [ ]:
# NC
output_table["what_tested"].append("bl_minus_darkest_nc")
output_table["test_type"].append("normality")
output_table["statistic"].append(nt_diff_nc.statistic)
output_table["p_val"].append(nt_diff_nc.pvalue)
if nt_diff_nc.pvalue < 0.05:
    output_table["comment"].append("dataset normality hypothesis rejected")
else:
    output_table["comment"].append("dataset normality hypothesis was not rejected")

### As normal data cannot be assumed, use Mann-Whitney

In [ ]:
mwu_bl_minus_darkest = mannwhitneyu(df_bl_minus_darkest_combined[df_bl_minus_darkest_combined["window_type"] == "ca1"]["bl-darkest"], df_bl_minus_darkest_combined[df_bl_minus_darkest_combined["window_type"] == "nc"]["bl-darkest"])
print(mwu_bl_minus_darkest)

In [ ]:
ttest_ind(df_bl_minus_darkest_combined[df_bl_minus_darkest_combined["window_type"] == "ca1"]["bl-darkest"], df_bl_minus_darkest_combined[df_bl_minus_darkest_combined["window_type"] == "nc"]["bl-darkest"], equal_var=False)

In [ ]:
# check SD category was recognized properly
fig = plt.figure()
for i in range(len(traces_ca1)):
    len_sz = len(traces_ca1[i]) -  n_am_frames_ca1[i] - n_bl_frames_ca1[i]
    plt.plot(traces_ca1[i][n_bl_frames_ca1[i]+len_sz+1:], color=colors_ca1[i])
plt.show()

In [ ]:
output_table["what_tested"].append("Bl_minus_darkest")
output_table["test_type"].append("Mann-Whitney U")
output_table["statistic"].append(mwu_bl_minus_darkest.statistic)
output_table["p_val"].append(mwu_bl_minus_darkest.pvalue)
if mwu_bl_minus_darkest.pvalue < 0.05:
    output_table["comment"].append("two datasets coming from same population rejected")
else:
    output_table["comment"].append("two datasets coming from same population not rejected")

## Export statistical tests to Excel

In [ ]:
df_stat_results = pd.DataFrame(output_table)

In [ ]:
if save_dsets:
    output_fpath = os.path.join(output_folder, f"recovery_stats_{get_datetime_for_fname()}.xlsx")
    df_stat_results.to_excel(output_fpath, index=False)

# Export data to excel

## 1. Recovery times

In [ ]:
if save_dsets:
    fpath_recovery_ca1 = os.path.join(output_folder, f"recovery_times_ca1_{get_datetime_for_fname()}.xlsx")
    df_recovery_ca1.to_excel(fpath_recovery_ca1, index=False)
    print(f"Saved file to {fpath_recovery_ca1}")
    
    fpath_recovery_nc = os.path.join(output_folder, f"recovery_times_nc_{get_datetime_for_fname()}.xlsx")
    df_recovery_nc.to_excel(fpath_recovery_nc, index=False)
    print(f"Saved file to {fpath_recovery_nc}")

In [ ]:
if save_dsets:
    fpath_recovery = os.path.join(output_folder, f"recovery_times_{get_datetime_for_fname()}.xlsx")
    df_recovery_combined.to_excel(fpath_recovery, index=False)
    print(f"Saved file to {fpath_recovery}")

## 2. Amplitudes

In [ ]:
if save_dsets:
    fpath_amplitudes = os.path.join(output_folder, f"sz_sd_amplitudes_{get_datetime_for_fname()}.xlsx")
    df_amplitudes_ca1[["mouse_id", "window_type", "event_uuid", "Sz-bl", "SD-bl" ]].sort_values(by="mouse_id").to_excel(fpath_amplitudes, index=False)
    print(f"Saved file to {fpath_amplitudes}")
    

## 3. Bl - darkest point differences

In [ ]:
if save_dsets:
    fpath_bl_darkest = os.path.join(output_folder, f"bl-to-darkest-point_{get_datetime_for_fname()}.xlsx")
    df_bl_minus_darkest_combined[["mouse_id", "event_uuid", "window_type", "baseline", "darkest_postictal", "bl-darkest",  ]].sort_values(by="mouse_id").to_excel(fpath_bl_darkest, index=False)
    print(f"Saved file to {fpath_bl_darkest}")